# Cross Validation & Update LB

In [ ]:
# https://zenn.dev/currypurin/scraps/47d5f84a0ca89d

import datetime
import time
from datetime import timezone

from kaggle.api.kaggle_api_extended import KaggleApi

api = KaggleApi()
api.authenticate()

COMPETITION = "ubiquant-market-prediction"
result_ = api.competition_submissions(COMPETITION)[0]
latest_ref = str(result_)  # 最新のサブミット番号
submit_time = result_.date

status = ""

while status != "complete":
    list_of_submission = api.competition_submissions(COMPETITION)
    for result in list_of_submission:
        if str(result.ref) == latest_ref:
            break
    status = result.status

    now = datetime.datetime.now(timezone.utc).replace(tzinfo=None)
    elapsed_time = int((now - submit_time).seconds / 60) + 1
    if status == "complete":
        print("\r", f"run-time: {elapsed_time} min, LB: {result.publicScore}")
    else:
        print("\r", f"elapsed time: {elapsed_time} min", end="")
        time.sleep(60)

In [1]:
import sys

sys.path.append("..")
sys.path.append("../../inputs")

In [2]:
import logging

logging.basicConfig(
    # filename=__file__.replace('.py', '.log'),
    level=logging.getLevelName("INFO"),
    format="%(asctime)s [%(levelname)s] [%(module)s] %(message)s",
)

log = logging.getLogger(__name__)

In [3]:
try:
    LB_SCORE = result.publicScore
except Exception:
    LB_SCORE = 0.150

In [9]:
from omegaconf import OmegaConf

c = OmegaConf.load("../config/main.yaml")

c.settings.debug = False
c.wandb.enabled = True
c.wandb.group = "LB"
c.wandb.dir = "../../cache/"
c.settings.dirs.working = ".."
c.settings.dirs.input = "../../inputs/"

pretrained = """
- dir: ../../datasets/trainings/2022-01-26_07-47-01/fold0/
  model: ump_1
- dir: ../../datasets/trainings/2022-01-26_07-47-03/fold1/
  model: ump_1
- dir: ../../datasets/trainings/2022-01-26_07-47-05/fold2/
  model: ump_1
- dir: ../../datasets/trainings/2022-01-26_07-47-07/fold3/
  model: ump_1
- dir: ../../datasets/trainings/2022-01-26_07-47-10/fold4/
  model: ump_1
- dir: ../../datasets/trainings/2022-01-26_07-47-12/fold5/
  model: ump_1
- dir: ../../datasets/trainings/2022-01-26_07-47-14/fold6/
  model: ump_1
"""

c.params.pretrained = OmegaConf.create(pretrained)

log.info(OmegaConf.to_yaml(c))

2022-01-26 21:42:21,246 [INFO] [2523262202] defaults:
- _self_
hydra:
  run:
    dir: ../outputs/${now:%Y-%m-%d_%H-%M-%S}
  job_logging:
    formatters:
      simple:
        format: '%(asctime)s [%(levelname)s][%(module)s] %(message)s'
wandb:
  enabled: true
  entity: imokuri
  project: ump
  dir: ../../cache/
  group: LB
settings:
  print_freq: 100
  gpus: 6,7
  dirs:
    working: ..
    input: ../../inputs/
  inputs:
  - train.csv
  - example_test.csv
  - example_sample_submission.csv
  debug: false
  n_debug_data: 100000
  amp: true
  multi_gpu: true
params:
  seed: 440
  n_class: 1
  n_fold: 7
  skip_training: false
  epoch: 20
  es_patience: 0
  batch_size: 640
  gradient_acc_step: 1
  max_grad_norm: 1000
  fold: group
  group_name: investment_id
  label_name: target
  dataset: ump_1
  model: ump_1
  pretrained:
  - dir: ../../datasets/trainings/2022-01-26_07-47-01/fold0/
    model: ump_1
  - dir: ../../datasets/trainings/2022-01-26_07-47-03/fold1/
    model: ump_1
  - dir: ../..

In [15]:
import pandas as pd
import os

import src.utils as utils
from src.get_score import record_result

In [11]:
run = utils.setup_wandb(c)

2022-01-26 21:42:21,514 [ERROR] [jupyter] Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: imokuri (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.9 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


2022-01-26 21:42:25,117 [INFO] [utils] WandB initialized. name: drawn-pond-29, id: 2v5mts7z


In [16]:
# TODO: simple_cpcv のときの oof の計算 0 で初期化して、足し算して、 5 で割る感じかな

oof_dfs = []

for n, training in enumerate(c.params.pretrained):
    oof_df = pd.read_feather(os.path.join(training.dir.rsplit("/", 2)[0], "oof_df.f"))
    oof_dfs.append(oof_df)

oof_df = pd.concat(oof_dfs)

record_result(c, oof_df, c.params.n_fold)

2022-01-26 21:42:56,896 [INFO] [get_score] Score: 0.13359


0.13358646953954795

In [17]:
run.summary["LB"] = LB_SCORE